In [3]:
import dask.dataframe as dd
import tensorflow as tf
import pandas as pd
import numpy as np
import dask.array as da
from dask.distributed import Client

In [81]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:62221")
client

/usr/local/lib/python3.7/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| msgpack | 1.0.2  | 1.0.3     | 1.0.3   |
| numpy   | 1.18.5 | 1.21.5    | 1.21.5  |
| pandas  | 1.3.4  | 1.3.5     | 1.3.5   |
| toolz   | 0.11.1 | 0.11.2    | 0.11.2  |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://127.0.0.1:62221' processes=4 threads=12, memory=16.00 GiB>

In [21]:
DATA_URL = '/Users/vignesh/Desktop/code/dask_data_proc/sample/data/iris.data.csv'
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
dtypes = {'sepal_length': np.float64, 'petal_width': np.float64, 'petal_length': np.float64, 'sepal_width': np.float64, 'class': np.str}
df = dd.read_csv(DATA_URL, names=features, dtype=dtypes)

In [22]:
big_df = dd.concat([df]*10000)

In [23]:
len(big_df)

1500000

In [24]:
big_df = big_df.repartition(npartitions=1)

In [25]:
big_df

,sepal_length,sepal_width,petal_length,petal_width,class
npartitions=1,,,,,
,float64,float64,float64,float64,object
,...,...,...,...,...


In [13]:
df_fut = big_df.persist()

In [14]:
df_fut

,sepal_length,sepal_width,petal_length,petal_width,class
npartitions=1,,,,,
,float64,float64,float64,float64,object
,...,...,...,...,...


In [26]:
big_df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [27]:
shuf = big_df.sample(frac=1)

In [28]:
res = shuf.compute()

In [19]:
import multiprocessing

multiprocessing.cpu_count()


12

In [20]:
multiprocessing.cpu_count()

12

In [29]:
res

,sepal_length,sepal_width,petal_length,petal_width,class
16,5.4,3.9,1.3,0.4,Iris-setosa
61,5.9,3.0,4.2,1.5,Iris-versicolor
81,5.5,2.4,3.7,1.0,Iris-versicolor
130,7.4,2.8,6.1,1.9,Iris-virginica
20,5.4,3.4,1.7,0.2,Iris-setosa
...,...,...,...,...,...
22,4.6,3.6,1.0,0.2,Iris-setosa
87,6.3,2.3,4.4,1.3,Iris-versicolor
81,5.5,2.4,3.7,1.0,Iris-versicolor
112,6.8,3.0,5.5,2.1,Iris-virginica


In [68]:
import glob
import csv

In [69]:
mnist_data_path = "/Users/vignesh/Downloads/archive/trainingSet/trainingSet/"
mnist_metadata_out_path = mnist_data_path + "metadata.csv"


In [70]:
data_rows = []

In [71]:
num_labels = 10

In [72]:
for i in range(num_labels):
    curr_label_path = mnist_data_path + str(i) + "/"
    jpgFilenamesList =  glob.glob(curr_label_path + '*.jpg')
    rows = [[str(fname), i] for fname in jpgFilenamesList]
    data_rows.extend(rows)
    print(curr_label_path, str(len(jpgFilenamesList)))

/Users/vignesh/Downloads/archive/trainingSet/trainingSet/0/ 4132
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/1/ 4684
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/2/ 4177
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/3/ 4351
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/4/ 4072
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/5/ 3795
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/6/ 4137
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/7/ 4401
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/8/ 4063
/Users/vignesh/Downloads/archive/trainingSet/trainingSet/9/ 4188


In [73]:
with open(mnist_metadata_out_path, 'w') as f: 
    write = csv.writer(f)
    write.writerows(data_rows) 


In [78]:
DATA_URL = mnist_metadata_out_path
feature_names = ["image_path", "label"]
dtypes = {'image_path': np.str, 'label': np.int16}
df = dd.read_csv(DATA_URL, names=feature_names, dtype=dtypes)

In [86]:
big_df = dd.concat([df]*1000)

In [87]:
len(big_df)

42000000

In [88]:
big_df.head()

,image_path,label
0,/Users/vignesh/Downloads/archive/trainingSet/t...,0
1,/Users/vignesh/Downloads/archive/trainingSet/t...,0
2,/Users/vignesh/Downloads/archive/trainingSet/t...,0
3,/Users/vignesh/Downloads/archive/trainingSet/t...,0
4,/Users/vignesh/Downloads/archive/trainingSet/t...,0


In [89]:
shuf = big_df.sample(frac=1)

In [90]:
shuf.head()

,image_path,label
28565,/Users/vignesh/Downloads/archive/trainingSet/t...,6
16966,/Users/vignesh/Downloads/archive/trainingSet/t...,3
16022,/Users/vignesh/Downloads/archive/trainingSet/t...,3
34816,/Users/vignesh/Downloads/archive/trainingSet/t...,8
40990,/Users/vignesh/Downloads/archive/trainingSet/t...,9


In [91]:
len(shuf)

42000000

In [94]:
workerDetails = client.scheduler_info()['workers']

In [96]:
big_df = big_df.repartition(npartitions=len(workerDetails))

In [97]:
big_df

,image_path,label
npartitions=4,,
,object,int16
,...,...
,...,...
,...,...
,...,...


In [105]:
len(big_df.partitions[0])

10500000

In [106]:
len(big_df.partitions[1])

10500000

In [107]:
len(big_df.partitions[2])

10500000

In [108]:
len(big_df.partitions[3])

10500000